# Development of Emotion and Reasoning in the General Speeches of the United Nations: A text-based machine learning approach
## Script 0: Data Cleaning, Preprocessing & Token Frequencies
### Author: Sarah Franzen

### Instructions BEFORE running this script:
- Ensure all required packages are installed. If not, set `InstallPackages = TRUE` (see code cells below).  
- Set your working directory appropriately.  
- The script will automatically create the required folder structure.  
- Later, you will be asked to download the data from:  
  https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/0TJX8Y  
  and store it **unzipped** inside the created folder *data_original*


### Description: 
- Extract documents from their original txt documents and store them as one csv
- Create new variables
    - year, country_code, country_name
    - speech_length_words
    - english_official_language
    - security_council_permanent
    - gender
    - position
    - speaker_name
    - Country (Year)
- Cleaning
    - remove line breaks, hypehnation etc.
- Preprocessing
    - remove punctuation, tokenize, lowercase, pure digit tokens, words shorter than 2 letters, POS-Tagging, stemm, stopword removal
    - create new variable: speech_length_preprocessd
- Word Frequencies
    - word counts of the preprocessed_corpus
    - count frequency of the dictionary words
    - calculate weighted frequency
- Final preprocessing (Not correct, I think)
    - Remove words that appear less than 10x times from the preprocessed corpus 

___

## Setup, Installation of required Packages and Libraries & Folder Structure

In [310]:
InstallPackages = False # Set this to True to install the following packages 

if InstallPackages:
    import sys

    packages = [
        "pandas",
        "nltk",
        "spacy",
        "numpy",
        "gensim",
        "pycountry",
        "wordcloud",
        "matplotlib",
        "tqdm",
        "seaborn"
    ]

    for package in packages:
        !{sys.executable} -m pip install {package}


DownloadAdditions = False # Set this to True to download these additional resources
if DownloadAdditions:
    nltk.download("punkt")
    nltk.download("averaged_perceptron_tagger")
    spacy.cli.download('en_core_web_lg') 

In [312]:
# == Import libraries for data processing and NLP ==

import gensim
import joblib
import nltk
import os
import pandas as pd
import pycountry
import random
import re
import spacy
import time
import pickle
import numpy as np

from collections import Counter
from itertools import chain
from multiprocessing import Pool, freeze_support
from nltk.stem import SnowballStemmer
from spacy.lang.en.stop_words import STOP_WORDS as SPACY_STOPWORDS
from string import punctuation
from tqdm import tqdm
from pathlib import Path

# === Initialize NLP Tools ===

translator = str.maketrans('', '', punctuation)
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])
stemmer = SnowballStemmer("english")
tagger = nltk.perceptron.PerceptronTagger()

In [313]:
# === Set Working Directory and create folder structure ===

# Prompt user to enter working directory path
#wd = input("Please enter your working directory path (e.g., C:\\Users\\sarah\\OneDrive\\Dokumente\\Masterarbeit): ").strip()

# Change to the entered working directory
#try:
   # os.chdir(wd)
    #print(f"Working directory set to: {os.getcwd()}")
#except FileNotFoundError:
   # print("ERROR: The directory you entered does not exist. Please restart and enter a valid path.")
    #exit(1)

# Set your working directory (adjust this as needed)
wd = r"C:\Users\sarah\OneDrive\Dokumente\Masterarbeit"
os.chdir(wd)

# Set base path to current working directory
base_path = Path.cwd()
data_path = base_path / "data"

# List of subfolders to create inside 'data'
subfolders = ["data_original", "dictionaries", "freq", "preprocessed", "temp"]

# Create 'data' folder if it doesn't exist
data_path.mkdir(exist_ok=True)

# Create subfolders
for name in subfolders:
    (data_path / name).mkdir(exist_ok=True)

print("\nFolder structure created:")
print(f"- {data_path}")
for name in subfolders:
    print(f"  - {name}")

# Prompt user to place raw data files
#print(f"\nPlease place your raw data files (unzipped) into the folder:\n  {data_path / 'data_original'}")
#input("Press Enter after you have placed the files to continue...")

#print("Continuing with the script...")



Folder structure created:
- C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data
  - data_original
  - dictionaries
  - freq
  - preprocessed
  - temp


In [314]:
# === Define Folder Paths ===

# If an error occurs, make sure that you actually have these folders in your working directory

data_c = wd / "data"
data_temp = data_c / "temp"
data_freq = data_c / "freq"
data_dict = data_c / "dictionaries"
data_preprocessed = data_c / "preprocessed"
fig_dir = wd /"fig"

In [315]:
print("Current working directory:", os.getcwd())

Current working directory: C:\Users\sarah\OneDrive\Dokumente\Masterarbeit


___

## Load and Prepare Corpus

In [322]:
# == Load the txt-files from the UN General Debate Corpus ==

# Set Folder path containing the original TXT files    
base_folder = r".\data\data_original\UN General Debate Corpus\UNGDC_1946-2023\TXT"

# Collect txt-files
all_txt_files = []
for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file.endswith('.txt') and not file.startswith('._'):
            all_txt_files.append(os.path.join(root, file))

print(f"Total speeches found: {len(all_txt_files)}")

# Randomly pick 800 files from the full collection   ################################################# REMOVE AT LATER POINT
sampled_files = random.sample(all_txt_files,10761)

# Read the selected files into a list
raw_data = []
for filepath in sampled_files:
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
        raw_data.append({'filename': os.path.basename(filepath), 'speech': content})

df_raw = pd.DataFrame(raw_data)

# Include only valid filenames
df_raw = df_raw[df_raw['filename'] != '.DS_Store-to-UTF-8.txt'].copy()

# Drop empty speeches
df_raw['speech'] = df_raw['speech'].astype(str)
df_raw = df_raw[df_raw['speech'].str.strip() != ''].copy()

# Check for duplicates
dupe_labels = df_raw[df_raw['filename'].duplicated(keep=False)]
print(dupe_labels[['filename', 'speech']].head(20))

# == Store as csv and pkl ==

raw_pickle_path = r".\data\un_corpus_raw.pkl"
df_raw.to_pickle(raw_pickle_path)

raw_output_path = r".\data\un_corpus_raw.csv"
df_raw.to_csv(raw_output_path, index=False, sep=';', encoding='utf-8')

print(f"\n Saved raw data with {len(df_raw)} speeches to '{raw_output_path}'")


Total speeches found: 10761
Empty DataFrame
Columns: [filename, speech]
Index: []

 Saved raw data with 10760 speeches to '.\data\un_corpus_raw.csv'


## Is this necessary?

In [324]:
# == Load data ==

df_raw = pd.read_pickle(r".\data\un_corpus_raw.pkl")
df_raw.head()         

,filename,speech
0,FRA_20_1965.txt,87. The French delegation does not conceal its...
1,SDN_43_1988.txt,﻿It gives me great pleasure to extend to you. ...
2,SGP_28_1973.txt,"﻿ 1. First of all, may I, on my own behalf and..."
3,MAR_28_1973.txt,"﻿83.\tMr. President, I am very pleased to be-a..."
4,LSO_51_1996.txt,﻿I wish to extend my\ndelegation’s sincere con...


___

## Create new variables

#### New Variables: Year, Country Code and Country Name

In [328]:
# == Create variable: country code & year ==

# Create contry_code and year variable
df_raw['country_code'] = df_raw['filename'].str.extract(r'^([A-Z]{2,3})')
df_raw['year'] = df_raw['filename'].str.extract(r'_(\d{4})\.txt$').astype(int)

print("Min year:", df_raw['year'].min())
print("Max year:", df_raw['year'].max())
# Speeches range from 1946 to 2023

# == Create variable: country_name by matching ISO country code 
code_to_name = {country.alpha_3: country.name for country in pycountry.countries}

# Add custom short names and legacy codes
custom_names = {
    "BOL": "Bolivia",
    "COD": "Democratic Republic of Congo",
    "IRN": "Iran",
    "LAO": "Laos",
    "MDA": "Moldova",
    "PRK": "North Korea",
    "PSE": "Palestine",
    "RUS": "Russia",
    "SYR": "Syria",
    "TZA": "Tanzania",
    "VAT": "Vatican City State",
    "VEN": "Venezuela",
    "VNM": "Vietnam",
    "YMD": "South Yemen",
    "YUG": "Yugoslavia",
    "DDR": "East Germany",
    "EU": "European Union",
    "CSK": "Czechoslovakia",
    "FSM": "Micronesia",
    "KOR": "South Korea"
}

code_to_name.update(custom_names)
df_raw['country_name'] = df_raw['country_code'].map(code_to_name)

# Check missing mappings
missing = df_raw.loc[df_raw['country_name'].isna(), 'country_code'].unique()
print("Missing codes:", missing)

Min year: 1946
Max year: 2023
Missing codes: []


In [329]:
# == Check country names and structure ==

pd.set_option('display.max_rows', None)
print(df_raw[['country_code', 'country_name']].drop_duplicates().sort_values('country_code').reset_index(drop=True))
pd.reset_option('display.max_rows')

    country_code                      country_name
0            AFG                       Afghanistan
1            AGO                            Angola
2            ALB                           Albania
3            AND                           Andorra
4            ARE              United Arab Emirates
5            ARG                         Argentina
6            ARM                           Armenia
7            ATG               Antigua and Barbuda
8            AUS                         Australia
9            AUT                           Austria
10           AZE                        Azerbaijan
11           BDI                           Burundi
12           BEL                           Belgium
13           BEN                             Benin
14           BFA                      Burkina Faso
15           BGD                        Bangladesh
16           BGR                          Bulgaria
17           BHR                           Bahrain
18           BHS               

#### New Variable: Length of speeches

In [331]:
# Count total number of unique tokens in the corpus
all_tokens = set()
for speech in df_raw['speech']:
    all_tokens.update(str(speech).split())
print("Total number of unique tokens in the corpus:", len(all_tokens))

# Count total number of tokens in the corpus
total_tokens = df_raw['speech'].apply(lambda x: len(str(x).split())).sum()
print("Total number of tokens in the corpus:", total_tokens)

# Add a new column: speech length in words
df_raw['speech_length_words'] = df_raw['speech'].apply(lambda x: len(str(x).split()))

# Calculate average length
avg_length = df_raw['speech_length_words'].mean()
print("Average speech length (words):", round(avg_length, 2))

# 20 shortest & longest speeches
print("20 shortest speeches:")
print(df_raw.nsmallest(20, 'speech_length_words')[['filename', 'country_name', 'year', 'speech_length_words']])

print("\n20 longest speeches:")
print(df_raw.nlargest(20, 'speech_length_words')[['filename', 'country_name', 'year', 'speech_length_words']])

Total number of unique tokens in the corpus: 217246
Total number of tokens in the corpus: 31541283
Average speech length (words): 2931.35
20 shortest speeches:
              filename                      country_name  year  \
1124   EGY_28_1973.txt                             Egypt  1973   
7615   VCT_41_1986.txt  Saint Vincent and the Grenadines  1986   
5608   BEN_71_2016.txt                             Benin  2016   
9334   GNB_76_2021.txt                     Guinea-Bissau  2021   
7809   HTI_01_1946.txt                             Haiti  1946   
5163   RWA_76_2021.txt                            Rwanda  2021   
2808   DDR_45_1990.txt                      East Germany  1990   
4456   LTU_73_2018.txt                         Lithuania  2018   
7112   RWA_69_2014.txt                            Rwanda  2014   
10533  IRN_01_1946.txt                              Iran  1946   
751    LTU_72_2017.txt                         Lithuania  2017   
7765   RWA_70_2015.txt                          

#### New variable: English as Official Language

In [333]:
# Source for english as official language : https://gradschool.utk.edu/future-students/office-of-graduate-admissions/applying-to-graduate-school/admission-requirements/testing-requirements/countries-with-english-as-official-language/
# They are quoting: https://www.cia.gov/the-world-factbook/field/languages/

english_countries = [
    "Anguilla", "Antigua and Barbuda", "Bahamas", "Barbados", "Belize", "Belgium",
    "Bermuda", "Botswana", "British Virgin Islands", "Burundi", "Cameroon", "Canada",
    "Cayman Islands", "Christmas Island", "Cook Islands", "Dominica", "Fiji", "Gambia",
    "Ghana", "Grenada", "Guyana", "Hong Kong", "India", "Ireland", "Jersey", "Kenya",
    "Liberia", "Malawi", "Malta", "Marshall Islands", "Micronesia",
    "Namibia", "New Zealand", "Nigeria", "Niue", "Norfolk Island", "Northern Mariana Islands",
    "Pakistan", "Palau", "Papua New Guinea", "Philippines", "Pitcairn Islands", "Rwanda",
    "Saint Kitts and Nevis", "Saint Lucia", "Samoa", "Seychelles", "Sierra Leone", "Singapore",
    "Sint Maarten", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Sudan",
    "Eswatini", "Tanzania", "Tonga", "Trinidad and Tobago", "Turks and Caicos Islands",
    "Tuvalu", "Uganda", "Zambia", "Zimbabwe"
]

# Create dummy column for english being the official language
df_raw['english_official_language'] = df_raw['country_name'].apply(
    lambda x: 1 if x in english_countries else 0
)

# Detect unmatched countries 
matched = set(df_raw['country_name'])
unmatched = [country for country in english_countries if country not in matched]

print("Countries not matched in df_raw['country_name']:")
for country in unmatched:
    print(country)

# All of these countries are either British Overseas Territories, Australian Territories, self-governing island territories or Special Administrative Regions
    # None of the unmatched regions are UN Members

# Check df with new variable english_official_language
df_raw.head()


Countries not matched in df_raw['country_name']:
Anguilla
Bermuda
British Virgin Islands
Cayman Islands
Christmas Island
Cook Islands
Hong Kong
Jersey
Niue
Norfolk Island
Northern Mariana Islands
Pitcairn Islands
Sint Maarten
Turks and Caicos Islands


,filename,speech,country_code,year,country_name,speech_length_words,english_official_language
0,FRA_20_1965.txt,87. The French delegation does not conceal its...,FRA,1965,France,4460,0
1,SDN_43_1988.txt,﻿It gives me great pleasure to extend to you. ...,SDN,1988,Sudan,3201,1
2,SGP_28_1973.txt,"﻿ 1. First of all, may I, on my own behalf and...",SGP,1973,Singapore,2790,1
3,MAR_28_1973.txt,"﻿83.\tMr. President, I am very pleased to be-a...",MAR,1973,Morocco,3086,0
4,LSO_51_1996.txt,﻿I wish to extend my\ndelegation’s sincere con...,LSO,1996,Lesotho,3148,0


#### New variable: Permanent member security council

In [335]:
# Define permanent members of the UN Security Council and create dummy
permanent_members = ['RUS', 'USA', 'FRA', 'GBR', 'CHN']

df_raw['security_council_permanent'] = df_raw['country_code'].isin(permanent_members).astype(int)

print(df_raw[df_raw['country_code'].isin(permanent_members)][
    ['country_code', 'country_name', 'security_council_permanent', 'year']
])

      country_code    country_name  security_council_permanent  year
0              FRA          France                           1  1965
61             FRA          France                           1  2014
97             CHN           China                           1  1978
183            RUS          Russia                           1  1982
197            FRA          France                           1  2012
...            ...             ...                         ...   ...
10665          FRA          France                           1  2008
10701          USA   United States                           1  2010
10704          RUS          Russia                           1  1966
10737          USA   United States                           1  1969
10758          GBR  United Kingdom                           1  1970

[383 rows x 4 columns]


#### New variables: Speaker, Position & Gender

In [337]:
# Load suplementary data set which contains information on the speakers name (sometimes including gender) and their position
df_speakers = pd.read_excel(os.path.join(data_c, "data_original", "UN General Debate Corpus", "Speakers_by_session.xlsx"))

# Check uniqueness of keys in df_speakers
print(df_speakers.duplicated(subset=['Year', 'ISO Code']).sum())

# Check for duplicates in df_speakers
dupes_speakers = df_speakers[df_speakers.duplicated(subset=['Year', 'ISO Code'], keep=False)]
print(dupes_speakers.sort_values(['Year', 'ISO Code']).head(20))

# For two observations the noted speakers differ, therefore an additional UN Resource was used to determine the real speaker
# for 1958 Iraq Mr. Jomard see https://digitallibrary.un.org/record/380721
# for 1954 Phillipines Mr. Romulo see https://digitallibrary.un.org/record/380429

# Overwrite the wrongfully noted speaker for Iraq and the Philipines and afterwards drop each of the double observation
df_speakers_cleaned = (
    df_speakers[~(
        ((df_speakers['ISO Code'] == "IRQ") & (df_speakers['Year'] == 1958) & (df_speakers['Name of Person Speaking'] == "Mr. Jawad")) |
        ((df_speakers['ISO Code'] == "PHL") & (df_speakers['Year'] == 1954) & (df_speakers['Name of Person Speaking'] == "Mr. SERRANO"))
    )]
    .drop_duplicates(subset=['Year', 'ISO Code'], keep='first')
)

# Check if there are duplicates that are still unaccounted for
print(df_speakers_cleaned.duplicated(subset=['Year', 'ISO Code']).sum())

17
       Year  Session ISO Code                              Country  \
10506  1951        6      RUS  Union of Soviet Socialist Republics   
10552  1951        6      RUS  Union of Soviet Socialist Republics   
10381  1954        9      PHL                          Philippines   
10415  1954        9      PHL                          Philippines   
10261  1956       11      IRQ                                 Iraq   
10324  1956       11      IRQ                                 Iraq   
10323  1956       11      SYR                                Syria   
10326  1956       11      SYR                                Syria   
10205  1957       12      CSK                       Czechoslovakia   
10243  1957       12      CSK                       Czechoslovakia   
10159  1958       13      BGR                             Bulgaria   
10181  1958       13      BGR                             Bulgaria   
10127  1958       13      CSK                       Czechoslovakia   
10183  1958      

In [338]:
# == Create variable speaker_name and position ==

# Merge new infrormation to dataframe
df_merged = df_raw.merge(
    df_speakers_cleaned[['Year', 'ISO Code', 'Name of Person Speaking', 'Post']],
    left_on=['year', 'country_code'],
    right_on=['Year', 'ISO Code'],
    how='left',
    indicator=True)

# Detect unmatched rows
unmatched = df_merged[df_merged['_merge'] == 'left_only']
unmatched_count = (df_merged['_merge'] == 'left_only').sum()

print(unmatched[['filename', 'year', 'country_code', 'country_name']])
print(f"{unmatched_count} rows could not be matched")

# Check if any duplicates were created
dupes_speakers = df_merged[df_merged.duplicated(subset=['year', 'country_code'], keep=False)]
print(dupes_speakers.sort_values(['year', 'country_code']).head(20))

# Clean up: 
#- Keep all rows, unmateched rows are being set to NA
#- Drop redundant columns and rename some columns
df_merged = df_merged.drop(columns=['Year', 'ISO Code', '_merge']).rename(columns={
    'Name of Person Speaking': 'speaker_name',
    'Post': 'position'})

# == Create gender dummy ==

df_merged['gender_dummy'] = df_merged['speaker_name'].apply(
    lambda name: 0 if pd.notnull(name) and re.search(r'^(?:Mr|Sir)\b', name, re.IGNORECASE)
    else 1 if pd.notnull(name) and re.search(r'^(?:Mrs|Ms)\b', name, re.IGNORECASE)
    else None)

# Count all values including NaN
counts = df_merged['gender_dummy'].value_counts(dropna=False)

# Build summary using .get() to handle missing keys
gender_summary = pd.DataFrame({
    'gender_dummy': ['0 (male)', '1 (female)', 'NaN (unknown)'],
    'count': [
        counts.get(0, 0),
        counts.get(1, 0),
        counts.get(np.nan, 0)
    ]
})

print(gender_summary)

              filename  year country_code        country_name
20     DOM_18_1963.txt  1963          DOM  Dominican Republic
107    SYR_18_1963.txt  1963          SYR               Syria
182    RUS_37_1982.txt  1982          RUS              Russia
209    PRT_72_2017.txt  2017          PRT            Portugal
216    SAU_18_1963.txt  1963          SAU        Saudi Arabia
...                ...   ...          ...                 ...
9355   GTM_26_1971.txt  1971          GTM           Guatemala
9440   IND_18_1963.txt  1963          IND               India
9958   CUB_18_1963.txt  1963          CUB                Cuba
10534  EGY_17_1962.txt  1962          EGY               Egypt
10754  YMD_34_1979.txt  1979          YMD         South Yemen

[84 rows x 4 columns]
84 rows could not be matched
Empty DataFrame
Columns: [filename, speech, country_code, year, country_name, speech_length_words, english_official_language, security_council_permanent, Year, ISO Code, Name of Person Speaking, Post, _me

In [339]:
# == Adjust position variable ==

print(df_merged['position'].unique())
# For speakers that have more than one position it appears that the higher position is always listed first, therefore the second position will be dropped in the following function

# Since there are many expression for the position variable and it dos not seem to be unified, the position variable is adjusted to enable more consitency

def normalize_position(pos):
    if pd.isna(pos):
        return pos

    pos = pos.strip()

    # Fix common typos and extra spaces
    pos = re.sub(r'\s+', ' ', pos) 
    pos_lower = pos.lower()

     # Turn all ministers that deal with foreign affairs and international relations to "Minister for Foreign Affairs"
    foreign_affairs_variants = [
        'minister for foregn affairs',
        'minister responsible for foreign affairs',
        'minsiter for foreign and caricom affairs',
        'minister for external affairs',
        'minister of external relations',  # <-- added
        'foreign minister',
        'minister for international affairs and cooperation',
        'minister for external relations',
        'federal minister for european and international affairs',
        'international cooperation',
        'federal minister for foreign affairs',
        'minister for foreign and caricom affairs',
        'minister of foreign affairs and cooperation',
        'minister for international relations and cooperation',
        'ministry of external relations',
        'acting minister for foreign affairs and international cooperation',
        'ministry of foreign affairs',
        'minister for foreign and political affairs',
        'federal minister for europe, integration, and foreign affairs',
        'federal minister for europe, integration and foreign affairs',
        'minister of foreign and european affaris',
        'minister of foreign affairs',
        'minister for foreign',
        'minister of foreign and european affairs and minister of immigration and asylum',
        'minister for foreign affairs and senegalese living abroad',
        'minister for foreign affairs with responsibility for brexit',
        'minister for foreign affairs and investment promotion'
       
    ]
    if any(variant in pos_lower for variant in foreign_affairs_variants):
        return "Minister for Foreign Affairs"

    # Fix "rime minister" typo
    pos = re.sub(r'(?i)\brime[- ]?minister\b', 'Prime Minister', pos)

    # Normalize different versions of Head of Government, President, Prime Minsiter and Vice-President-
    exact_matches = {
        r'(?i)^president of (the )?government$': 'Head of Government',
        r'(?i)^acting president$': 'President',
        r'(?i)^interim president$': 'President',
        r'(?i)^constitutional president$': 'President',
        r'(?i)^first executive president$': 'President',
        r'(?i)^first prime[- ]?minister$': 'Prime Minister',
        r'(?i)^head of the goverment$': 'Head of Government',  # <-- catch typo + spaces
        r'(?i)^head\s+of\s+govern?ment$': 'Head of Government',
        r'(?i)^first vice[- ]?president$': 'Vice-President'
    }
    for pattern, replacement in exact_matches.items():
        if re.fullmatch(pattern, pos):
            return replacement

    # Normalize prefixes
    pos = re.sub(r'(?i)^first vice[- ]?president\b', 'Vice-President', pos)
    pos = re.sub(r'(?i)\bprime[- ]?minister\b', 'Prime Minister', pos)
    pos = re.sub(r'(?i)\bpresident\b', 'President', pos)
    pos = re.sub(r'(?i)\bvice[- ]?president\b', 'Vice-President', pos)

    # Collapse primary roles if they appear at start
    primary_roles = [
        (r'(?i)^prime[- ]?minister\b', 'Prime Minister'),
        (r'(?i)^deputy prime[- ]?minister\b', 'Deputy Prime Minister'),
        (r'(?i)^president\b', 'President'),
        (r'(?i)^vice[- ]?president\b', 'Vice-President'),
        (r'(?i)^head of state\b', 'Head of State'),
        (r'(?i)^(crown prince|prince|king|emir|amir)\b', 'Monarch'),
        (r'(?i)^(un representative|permanent representative|delegation|chair of (the )?delegation|chair of diplomatic representative)\b', 'Diplomatic Representative')
    ]
    for pattern, replacement in primary_roles:
        if re.match(pattern, pos):
            return replacement

    # Monarchs
    if re.search(r'(?i)\b(crown prince|prince|king|emir|amir)\b', pos):
        return "Monarch"

    # Head of State
    if re.search(r'(?i)head of state', pos):
        return "Head of State"
        
    # Diplomatic Representatives
    if re.search(r'(?i)(un representative|permanent representative|delegation|chair of (the )?delegation|chair of diplomatic representative)', pos):
        return "Diplomatic Representative"

    # Not indicated
    if re.search(r'(?i)not indicated', pos):
        return np.nan

    # Everything that is leftover becomes Others
    print("Unmatched position:", pos) 
    return "Others"


df_merged["position"] = df_merged["position"].apply(normalize_position)

[nan 'UN Representative' 'Deputy Prime Minister' 'Vice-President'
 'Minister for Foreign Affairs' 'Prime Minister'
 'Minister for Foreign Affairs and CARICOM Affairs' 'President'
 'Minister for Foreign Affairs and International Development Coperation'
 'Head of Government' 'Chair of Delegation' 'President ' 'Prime minister'
 'Vice-President ' 'Minister of Foreign Affairs ' 'Head of State'
 'Ministry of Foreign Affairs '
 'Prime Minister and Minister of Defence, Economic Growth and Job Creation'
 'Chair of the Delegation'
 'Prime Minister and Minister for Foreign Affairs'
 'Minister for Foregn Affairs' 'Emir'
 'Minister for Foreign Affairs and Foreign Trade' 'vice-President'
 'Acting Minister for Foreign Affairs and International Cooperation'
 'Federal Minister for Foreign Affairs'
 'Federal Minister for Europe, Integration, and Foreign Affairs'
 'Vice President' 'Head of state'
 'Minister for Foreign Affairs, International Business and Diaspora Relations'
 'Minister for International R

In [340]:
# Standardize position titles by merging deputy, second, and vice roles into their corresponding official positions for consistent categorization

def merge_positions(pos):
    if pd.isna(pos):
        return pos 
    
    if pos in ["Prime Minister", "Deputy Prime Minister"]:
        return "(Deputy) Prime Minister"
    
    if pos in ["President", "Vice-President"]:
        return "(Vice-) President"
        
    if pos in ["Minister for Foreign Affairs", "Deputy Minister for Foreign Affairs",
        "Deputy Minister Foreign Affairs",
        "Second Minister for Foreign Affairs",
        "Second Minister for Foreign Affairs and Trade",
        "Vice Minister for Foreign Affairs"]:
        return "(Deputy) Minister for Foreign Affairs"
    
    return pos

df_merged["position"] = df_merged["position"].apply(merge_positions)

In [341]:
pd.set_option("display.max_rows", None)
position_counts = df_merged['position'].value_counts(dropna=False)
print(position_counts)

NaN                                      4679
(Deputy) Minister for Foreign Affairs    2332
(Vice-) President                        1988
(Deputy) Prime Minister                  1190
Diplomatic Representative                 333
Head of State                              74
Others                                     60
Monarch                                    56
Head of Government                         48
Name: position, dtype: int64


In [342]:
# Started to document positions properly from 1986 on, before yearly sample size per year mostly less than 20 samples

yearly_counts = df_merged.groupby('year')['position'].agg(
    total_rows='size',
    missing=lambda x: x.isna().sum()
)

# Add not_missing column
yearly_counts['not_missing'] = yearly_counts['total_rows'] - yearly_counts['missing']

# Print the entire table
pd.set_option('display.max_rows', None)  # show all rows
print(yearly_counts)
pd.reset_option('display.max_rows')

      total_rows  missing  not_missing
year                                  
1946          39       39            0
1947          39       39            0
1948          39       39            0
1949          35       35            0
1950          44       44            0
1951          51       51            0
1952          43       43            0
1953          44       44            0
1954          42       41            1
1955          45       45            0
1956          66       66            0
1957          71       71            0
1958          72       72            0
1959          79       78            1
1960          79       60           19
1961          81       81            0
1962          92       90            2
1963          97       95            2
1964          95       93            2
1965         101      101            0
1966         108      108            0
1967         110      109            1
1968         112      110            2
1969         116      113

### New Variable: Country (Year)

This variable is later needed to create clean description plots and tables

In [344]:
df_merged = df_merged.copy()
df_merged['speech_label'] = df_merged['country_name'] + " (" + df_merged['year'].astype(str) + ")"

### Save dataframe with all new variables as un_corpus_merged

In [346]:
os.chdir(wd)

merged_pickle_path = r".\data\un_corpus_merged.pkl"
df_merged.to_pickle(merged_pickle_path)
 
merged_output_path = r".\data\un_corpus_merged.csv"
df_merged.to_csv(merged_output_path, index=False, sep=';', encoding='utf-8')

___

## Cleaning & Pre-processing

#### Cleaning

In [350]:
# == Clean corpus by removing empty spaces, line breaks, hyphenation, stray characters, and escape quote ==

def cleaning(content):
    if pd.isna(content):
        return ""
    
    content = content.replace('\n', ' ').replace('\r', ' ')
    content = ' '.join(content.split())

    # Ensure spacing after punctuation
    content = re.sub(r'(?<=[.,])(?=[^\s])', r' ', content)

    # Remove hyphenation at line breaks (e.g. "inter- national" → "international")
    content = re.sub(r'-\s', '', content)

    # Replace hyphen between letters with a space to prevent merging words (e.g. "russian-and" → "russian and")
    content = re.sub(r'(?<=\w)-(?=\w)', ' ', content)

    content = content.replace("\\", "")

    return content

df_merged['speech'] = df_merged['speech'].astype(str)  # Ensure column is string type
df_clean = df_merged.copy()
df_clean['speech'] = df_clean['speech'].apply(cleaning)

# Drop rows with empty speeches after cleaning
df_clean = df_clean[df_clean['speech'].str.strip().astype(bool)].reset_index(drop=True)


In [351]:
# == Split cleaned data into chunks and save as separate files ==

clean_data = df_clean[['filename', 'speech']].values.tolist()

data_id1 = clean_data[:int(len(clean_data)/4)]
data_id2 = clean_data[int(len(clean_data)/4): int(2*len(clean_data)/4)]
data_id3 = clean_data[int(2*len(clean_data)/4): int(3*len(clean_data)/4)]
data_id4 = clean_data[int(3*len(clean_data)/4):]

os.chdir(data_temp)

joblib.dump(data_id1, 'clean_speeches_indexed1.pkl')
joblib.dump(data_id2, 'clean_speeches_indexed2.pkl')
joblib.dump(data_id3, 'clean_speeches_indexed3.pkl')
joblib.dump(data_id4, 'clean_speeches_indexed4.pkl')

# Store list of cleaned data chunk paths to feed into preprocessing function later
clean_files = [
    os.path.join(data_temp, 'clean_speeches_indexed1.pkl'),
    os.path.join(data_temp, 'clean_speeches_indexed2.pkl'),
    os.path.join(data_temp, 'clean_speeches_indexed3.pkl'),
    os.path.join(data_temp, 'clean_speeches_indexed4.pkl')
]

print(f"Saved clean speeches chunks in '{data_temp}'")

Saved clean speeches chunks in 'C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp'


### Text Pre-Processing

#### Extend stopwords list

In [370]:
# Full path to stopwords pickle
stopwords_path = os.path.join(data_c, "stopwords.pkl")

# Load & sort stopwords
stopwords = joblib.load(stopwords_path)
stopwords = sorted(stopwords)

print(f"Loaded {len(stopwords)} stopwords (sorted alphabetically)")
print(stopwords[:300])

Loaded 18997 stopwords (sorted alphabetically)
['88', 'a', 'a.', 'aandahl', 'aaron', 'aaronsburg', 'aarp', 'abac', 'abbevill', 'abbitt', 'abbot', 'abbotsford', 'abbott', 'abbottstown', 'abbyvill', 'abdnor', 'abe', 'abel', 'abercrombi', 'aberdeen', 'abern', 'abernathi', 'abernethi', 'abi', 'abiel', 'abijah', 'abilen', 'abingdon', 'abington', 'abiquiu', 'abmp', 'abner', 'abourezk', 'about', 'abov', 'abraham', 'abram', 'absalom', 'absaraka', 'absaroke', 'absecon', 'abzug', 'acadia', 'acampo', 'accid', 'accokeek', 'accomac', 'accomack', 'accord', 'accovill', 'ace', 'acevedo-vilã¡', 'acheson', 'achill', 'acker', 'ackerman', 'ackermanvill', 'acklen', 'ackley', 'ackworth', 'acm', 'acosta', 'acra', 'acton', 'acushnet', 'acworth', 'ada', 'adah', 'adair', 'adairsvill', 'adairvill', 'adak', 'adam', 'adamsburg', 'adamson', 'adamstown', 'adamsvill', 'addabbo', 'addam', 'addi', 'addievill', 'addington', 'addison', 'addonizio', 'addyston', 'adel', 'adelanto', 'adelbert', 'adelphi', 'adelphia', 'adena

In [155]:
# Get SpaCy stopwords
#SPACY_STOPWORDS = list(nlp.Defaults.stop_words)
# Save stopwords
#joblib.dump(SPACY_STOPWORDS, stopwords_path)

#print(f"Saved {len(SPACY_STOPWORDS)} stopwords to {stopwords_path}")

In [156]:
# == Functions to remove punctioation, tokenize, lowercase, pure digit tokens, words shorter than 2 letters, POS-Tagging, stemm, stopword removal ==

def pro1(lista):
    return [[row[0], row[1].translate(translator)] for row in lista]

def pro2(lista):
    return [[row[0], gensim.utils.simple_preprocess(row[1])] for row in lista]

def pro3(lista):
        a = [[row[0], [w for w in row[1] if not w.isdigit()]] for row in lista]
        return a
    
def pro4(lista):
    return [[row[0], [w for w in row[1] if len(w) > 2]] for row in lista]


def tags(lista):
    t = [[row[0], tagger.tag(row[1])] for row in lista]  # tag each tokenlist
    t = [[row[0], [i[0] for i in row[1] if i[1].startswith(('N', 'V', 'J'))]] for row in t]
    return t
    
def pro5(lista):
    return [
        [row[0], [stemmer.stem(token) for token in row[1]]]
        for row in lista
    ]
    
def pro6(lista):
    return [[row[0], [w for w in row[1] if w not in stopwords]] for row in lista]
      
########################## Question for Max: They removed procedural words in the paper

def dropnull(lista):
    return [row for row in lista if len(' '.join(row[1])) > 0]

In [157]:
# == Create full pre-processing function and call it

def preprocessing(data_name):
    t0 = time.time()
    print(f"Starting preprocessing for {data_name}...")

    data = joblib.load(data_name)
    data = pro1(data)
    data = pro2(data)
    data = pro3(data)
    data = pro4(data)

    print(f"[{data_name}] Before tagging: {time.time() - t0:.2f}s")
    data = tags(data)
    print(f"[{data_name}] After tagging: {time.time() - t0:.2f}s")

    data = pro5(data)
    data = pro6(data)
    
    data = dropnull(data)

    filename_preprocessed = data_name.replace('clean_speeches_', 'preprocessed_speeches_').replace('.pkl', '.pkl')
    out_preprocessed = os.path.join(data_preprocessed, os.path.basename(filename_preprocessed))
    joblib.dump(data, out_preprocessed
               )
    print(f"[{data_name}] Saved stemmed version: {out_preprocessed}")

    print(f"[{data_name}] Done. Total time: {time.time() - t0:.2f}s\n")

def main():
    for fname in clean_files:
        preprocessing(fname)

if __name__ == "__main__":
    main()

Starting preprocessing for C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed1.pkl...
[C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed1.pkl] Before tagging: 24.52s
[C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed1.pkl] After tagging: 465.86s
[C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed1.pkl] Saved stemmed version: C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\preprocessed\preprocessed_speeches_indexed1.pkl
[C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed1.pkl] Done. Total time: 2659.51s

Starting preprocessing for C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed2.pkl...
[C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed2.pkl] Before tagging: 25.06s
[C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed2.pkl] After tagging: 381.66s
[C:\U

In [382]:
# Store the pre-processed data
preprocessed_files = [
    os.path.join(data_preprocessed, 'preprocessed_speeches_indexed1.pkl'),
    os.path.join(data_preprocessed, 'preprocessed_speeches_indexed2.pkl'),
    os.path.join(data_preprocessed, 'preprocessed_speeches_indexed3.pkl'),
    os.path.join(data_preprocessed, 'preprocessed_speeches_indexed4.pkl')
]

In [407]:
# Load all preprocessed pickle files
preprocessed_data = []
for f in preprocessed_files:
    preprocessed_data.extend(joblib.load(f))

# Turn into DataFrame
df_preprocessed = pd.DataFrame(preprocessed_data, columns=["filename", "speech_preprocessed"])

# Merge into df_merged
df_merged = df_merged.merge(df_preprocessed, on="filename", how="left")

print(df_merged.head())


          filename                                speech_preprocessed
0  FRA_20_1965.txt  [deleg, conceal, satisfact, crisi, prevent, ge...
1  SDN_43_1988.txt  [give, great, pleasur, extend, sir, congratul,...
2  SGP_28_1973.txt  [behalf, deleg, extend, felicit, presid, unani...
3  MAR_28_1973.txt  [presid, pleas, abl, add, congratul, moroccan,...
4  LSO_51_1996.txt  [wish, extend, deleg, sincer, congratul, sir, ...


In [417]:
# == New variable: Speech length of the preprocessed corpus ==

# Count tokens in preprocessed speech
df_merged["speech_length_preprocessed"] = df_merged["speech_preprocessed"].apply(
    lambda x: len(x) if isinstance(x, list) else 0
)

# Quick check
print(df_merged[["filename", "speech_length_preprocessed"]].head())
all_tokens = [token for speech in df_merged["speech_preprocessed"].dropna() for token in speech]
unique_tokens = set(all_tokens)
print("Total unique tokens:", len(unique_tokens))

# Average length of preprocessed speeches
average_length = df_merged["speech_length_preprocessed"].mean()

print(f"Average number of tokens per speech: {average_length:.2f}")

          filename  speech_length_preprocessed
0  FRA_20_1965.txt                        1473
1  SDN_43_1988.txt                        1314
2  SGP_28_1973.txt                         978
3  MAR_28_1973.txt                        1009
4  LSO_51_1996.txt                        1210
Total unique tokens: 36093
Average number of tokens per speech: 1083.60


In [411]:
print(df_merged.head())

          filename                                             speech  \
0  FRA_20_1965.txt  87. The French delegation does not conceal its...   
1  SDN_43_1988.txt  ﻿It gives me great pleasure to extend to you. ...   
2  SGP_28_1973.txt  ﻿ 1. First of all, may I, on my own behalf and...   
3  MAR_28_1973.txt  ﻿83.\tMr. President, I am very pleased to be-a...   
4  LSO_51_1996.txt  ﻿I wish to extend my\ndelegation’s sincere con...   

  country_code  year country_name  speech_length_words  \
0          FRA  1965       France                 4460   
1          SDN  1988        Sudan                 3201   
2          SGP  1973    Singapore                 2790   
3          MAR  1973      Morocco                 3086   
4          LSO  1996      Lesotho                 3148   

   english_official_language  security_council_permanent  \
0                          0                           1   
1                          1                           0   
2                          1    

In [384]:
print(os.getcwd())

C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp


---

## Word-Frequencies

### Count frequencies of all tokens and display the most common words

In [390]:
#== Count token frequencies ==

def count_frequencies(filenames):
    total_freq = Counter()
    for fname in tqdm(filenames):
        data = joblib.load(fname)
        tokens = chain.from_iterable(row[1] for row in data if isinstance(row[1], list))
        total_freq.update(tokens)
    return total_freq

#def remove_rare_words(filenames, freqs, min_count=10):
   # for fname in filenames:
       # data = joblib.load(fname)
       # filtered_data = []
        #for doc_id, tokens in data:
          #  filtered_tokens = [w for w in tokens if freqs.get(w, 0) >= min_count]
          #  filtered_data.append([doc_id, filtered_tokens])
       # joblib.dump(filtered_data, fname)  # overwrite or save as new file
       # print(f"Processed {fname}: removed words with freq < {min_count}")

# === Count for preprocessed (stemmed) speeches ===
word_counts = count_frequencies(preprocessed_files)

#remove_rare_words(preprocessed_files, word_counts, min_count=10)

print("\n[Stemmed] Top 50 most common words:")
for word, count in word_counts.most_common(50):
    print(f"{word}: {count}")

print("\n[Stemmed] Top 50 least common words:")
for word, count in word_counts.most_common()[-50:]:
    print(f"{word}: {count}")

# Save stemmed word counts
save_path = os.path.join(data_freq, 'word_counts.pkl')
joblib.dump(word_counts, save_path)

100%|██████████| 4/4 [00:18<00:00,  4.73s/it]



[Stemmed] Top 50 most common words:
nation: 232161
countri: 176281
intern: 159427
develop: 145322
world: 131639
peopl: 126138
secur: 84573
general: 76545
govern: 74944
econom: 72976
right: 66637
year: 65809
assembl: 58038
problem: 56698
human: 56089
support: 54698
continu: 53008
communiti: 48791
region: 48260
polit: 48051
time: 46915
need: 40740
import: 40611
achiev: 38999
make: 37608
conflict: 36930
situat: 36044
principl: 35855
presid: 35771
resolut: 34924
global: 34853
take: 34787
africa: 34467
great: 33901
relat: 33670
oper: 33290
order: 33261
concern: 33087
action: 32165
nuclear: 32063
solut: 31810
establish: 31473
confer: 31242
made: 31165
polici: 30489
social: 30420
effect: 30030
respect: 29992
commit: 29893
part: 29740

[Stemmed] Top 50 least common words:
gon: 1
downto: 1
troubadour: 1
portau: 1
bloweth: 1
listeth: 1
carcinogen: 1
cadav: 1
erasmuss: 1
crossparti: 1
bycas: 1
ultraracist: 1
emarati: 1
allethiopia: 1
wardisplac: 1
merihan: 1
brittan: 1
aquilino: 1
sheitaat: 1
kl

['C:\\Users\\sarah\\OneDrive\\Dokumente\\Masterarbeit\\data\\freq\\word_counts.pkl']

In [391]:
num_unique_words = len(word_counts)
print(f"Number of unique words: {num_unique_words}")

Number of unique words: 36093


In [392]:
os.chdir(data_c)
print(os.getcwd())

C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data


### Count the frequency of the dictionary words

In [394]:
# == Count dictionary words

# Load dictionaries            ##### How did they come up with this dictionary? Why did they exclude words?

affect_path = os.path.join(data_dict, 'dictionary_affect.pkl')
cognition_path = os.path.join(data_dict, 'dictionary_cognition.pkl')

affect = joblib.load(affect_path)
cognition = joblib.load(cognition_path)

print("Contents of affect dictionary:")
print(affect)
print("Number of words in affect dictionary:", len(affect))

print("\nContents of cognition dictionary:")
print(cognition)
print("Number of words in cognition dictionary:", len(cognition))

a = [[i, word_counts[i]] for i in affect if i in word_counts]
c = [[i, word_counts[i]] for i in cognition if i in word_counts]

a = sorted(a, key=lambda x: x[1], reverse=True)
c = sorted(c, key=lambda x: x[1], reverse=True)

a = [[i[0], f"({i[1]}),"] for i in a]
c = [[i[0], f"({i[1]}),"] for i in c]

a1 = ' '.join(str(r) for v in a for r in v)
c1 = ' '.join(str(r) for v in c for r in v)

affect_out_path = os.path.join(data_freq, "affect_words.txt")
cog_out_path = os.path.join(data_freq, "cog_words.txt")

os.makedirs(data_freq, exist_ok=True)  # ensure directory exists

with open(affect_out_path, "w") as output:
    output.write(a1)

with open(cog_out_path, "w") as output:
    output.write(c1)


# == Calculate weighted frequencies for all words

l = sum(word_counts.values())

a = 0.001 # Method to downweight with a smoothing parameter: For frequent words (large v/1), weight approaches 0; for rare words (small v/1) closer to 1
word_counts_weighted = {k: a / (a + (v / l)) for k, v in word_counts.items()}

joblib.dump(word_counts_weighted, os.path.join(data_freq, 'word_counts_weighted.pkl'))

################################################################################ ISSUE##################
# To print top 100 by weighted values, sort the dictionary by value descending:
top_100_weighted = sorted(word_counts_weighted.items(), key=lambda x: x[1], reverse=True)[:100]

print("Top 100 words by weighted frequency:")
for word, weight in top_100_weighted:
    print(f"{word}: {weight}")

Contents of affect dictionary:
['forbid', 'unattract', 'cruelti', 'crappi', 'apathi', 'scari', 'unimpress', 'sin', 'dumbest', 'eas', 'agit', 'sob', 'shocker', 'tragedi', 'fabul', 'strongest', 'giver', 'sigh', 'aw', 'witch', 'hurtl', 'fucktard', 'cruel', 'glamor', 'funni', 'smarter', 'brillianc', 'irrate', 'alright', 'honest', 'profit', 'fearless', 'grievous', 'relax', 'isolationist', 'hah', 'shyness', 'poorest', 'cruelest', 'troublemak', 'disagre', 'agon', 'terror', 'fight', 'pleas', 'poor', 'crazi', 'hostil', 'stupid', 'damnat', 'vain', 'jade', 'heartless', 'nag', 'gloomi', 'damn', 'dishearten', 'pleaser', 'credit', 'warmth', 'greatest', 'whine', 'shame', 'angriest', 'envious', 'grin', 'blameless', 'sweeter', 'laidback', 'stupidest', 'unprotect', 'whiner', 'unlov', 'shake', 'boredom', 'fairer', 'weaker', 'wellb', 'bold', 'sucki', 'unsuccess', 'mourner', 'liken', 'defens', 'invigor', 'tedious', 'paranoid', 'cynic', 'dignifi', 'paranoia', 'sweetest', 'contented', 'humili', 'crush', 'ter

---

## Final Cleaning

In [427]:
os.chdir(data_freq)

word_counts = joblib.load('word_counts.pkl')  # load stemmed counts
# For each speech only keep tokens that appear at least 10x

def select(lista):
    for i in range(len(lista)):
        x = lista[i][0]
        y = lista[i][1]
        y = [w for w in y if count.get(w, 0) >= 10]
        lista[i] = [x, y]
    return lista

for data_path in preprocessed_files:
    data = joblib.load(data_path)
    data = select(data)
    cleaned_path = data_path.replace('.pkl', '_final.pkl')
    joblib.dump(data, cleaned_path)

In [429]:
os.chdir(data_preprocessed)

final_files = [
    os.path.join(data_preprocessed, 'preprocessed_speeches_indexed1_final.pkl'),
    os.path.join(data_preprocessed, 'preprocessed_speeches_indexed2_final.pkl'),
    os.path.join(data_preprocessed, 'preprocessed_speeches_indexed3_final.pkl'),
    os.path.join(data_preprocessed, 'preprocessed_speeches_indexed4_final.pkl')
]

final_data = []
for fname in final_files:
    final_data.extend(joblib.load(fname))

# Merge with df_merged
df_final = pd.DataFrame(final_data, columns=["filename", "speech_final"])
df_merged = df_merged.merge(df_final, on="filename", how="left")

# Create speech_length_final column
df_merged["speech_length_final"] = df_merged["speech_final"].apply(
    lambda x: len(x) if isinstance(x, list) else 0
)

# Quick check
print(df_merged[["filename", "speech_length_final"]].head())

all_tokens_final = [token for speech in df_merged["speech_final"].dropna() for token in speech]
unique_tokens_final = set(all_tokens_final)
print("Total unique tokens across all final speeches:", len(unique_tokens_final))

print("Average tokens per final speech:", df_merged["speech_length_final"].mean())


          filename  speech_length_final
0  FRA_20_1965.txt                 1471
1  SDN_43_1988.txt                 1312
2  SGP_28_1973.txt                  969
3  MAR_28_1973.txt                  999
4  LSO_51_1996.txt                 1210
Total unique tokens across all final speeches: 10951
Average tokens per final speech: 1078.4774163568773
